#### Implementation Part is at the start of Notebook, VIsualisation part is at the End.

In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



## Libraries

In [ ]:

# standard
#import numpy as np
#import pandas as pd
#import time

# plots
#import matplotlib.pyplot as plt
#import seaborn as sns

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder

from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier


## Data Import

In [ ]:
# load data + first glance
#train = pd.read_csv('../input/tabular-playground-series-mar-2021/train.csv')
#test = pd.read_csv('../input/tabular-playground-series-mar-2021/test.csv')

# first glance (training data)
#train.head()
LOCAL_FLAG = False

INPUT_PATH = '/kaggle/input/tabular-playground-series-mar-2021/'
TRAIN_FILE = 'train.csv'
TEST_FILE = 'test.csv'
SUBMISSION_FILE = 'sample_submission.csv'


EARLY_STOPPING_ROUNDS = 200
NUM_FOLDS = 10
SEED = 42


In [ ]:


#cont_features = [
 #   "cont0", "cont1", "cont2", "cont3", "cont4", "cont5", "cont6", "cont7",
  #  "cont8", "cont9", "cont10",
#]
#cat_features = [
 #   "cat0", "cat1", "cat2", "cat3", "cat4", "cat5", "cat6", "cat7",
  #  "cat8", "cat9", "cat10", "cat11", "cat12", "cat13", "cat14", "cat15",
  #  "cat16", "cat17", "cat18"
#]
#target = train["target"]

train = pd.read_csv(INPUT_PATH + TRAIN_FILE, index_col='id')
test = pd.read_csv(INPUT_PATH + TEST_FILE, index_col='id')
submission = pd.read_csv(INPUT_PATH + SUBMISSION_FILE, index_col='id')



In [ ]:
print(train.nunique())

In [ ]:
#from category_encoders import LeaveOneOutEncoder
#from sklearn.preprocessing import LabelEncoder

#xgb_cat_features = []
#lgb_cat_features = []
#cb_cat_features = []
#ridge_cat_features = []
#hgbc_cat_features = []

#loo_features = []
#le_features = []

#def label_encode(train_df, test_df, column):
 #   le = LabelEncoder()
  #  new_feature = "{}_le".format(column)
   # le.fit(train_df[column].unique().tolist() + test_df[column].unique().tolist())
   # train_df[new_feature] = le.transform(train_df[column])
   # test_df[new_feature] = le.transform(test_df[column])
   # return new_feature

#def loo_encode(train_df, test_df, column):
 #   loo = LeaveOneOutEncoder()
  #  new_feature = "{}_loo".format(column)
   # loo.fit(train_df[column], train_df["target"])
    #train_df[new_feature] = loo.transform(train_df[column])
    #test_df[new_feature] = loo.transform(test_df[column])
    #return new_feature

#for feature in cat_features:
 #   loo_features.append(loo_encode(train, test, feature))
  #  le_features.append(label_encode(train, test, feature))
    
#xgb_cat_features.extend(loo_features)
#lgb_cat_features.extend(le_features)
#cb_cat_features.extend(cat_features)
#ridge_cat_features.extend(loo_features)
#hgbc_cat_features.extend(loo_features)


for c in train.columns:
    if train[c].dtype=='object': 
        lbl = LabelEncoder()
        lbl.fit(list(train[c].values) + list(test[c].values))
        train[c] = lbl.transform(train[c].values)
        test[c] = lbl.transform(test[c].values)
        
train.head()

In [ ]:
target = train.pop('target')

In [ ]:
target = target.to_numpy()

In [ ]:
target.shape

In [ ]:
columns = test.columns

In [ ]:
print(columns)

In [ ]:
train = pd.read_csv(INPUT_PATH + TRAIN_FILE, index_col='id')
test = pd.read_csv(INPUT_PATH + TEST_FILE, index_col='id')

for c in train.columns:
    if train[c].dtype=='object': 
        lbl = LabelEncoder()
        lbl.fit(list(train[c].values) + list(test[c].values))
        train[c] = lbl.transform(train[c].values)
        test[c] = lbl.transform(test[c].values)

        
target = train.pop('target')
target = target.to_numpy()

In [ ]:
def cross_valid(model, train, target, num_folds=10, random_state=42):#, init_model=pretrain_lgbm):

    train_oof = np.zeros((len(train)))
    test_preds = 0

    kf = StratifiedKFold(n_splits=num_folds, random_state=SEED, shuffle=True)
    aucs=[]

    for f, (train_ind, val_ind) in tqdm(enumerate(kf.split(train, target))):

        train_df, val_df = train.iloc[train_ind][columns], train.iloc[val_ind][columns]
        
        train_target, val_target = target[train_ind], target[val_ind]

        model.fit(train_df, 
        train_target,
        eval_set= [(train_df, train_target), (val_df, val_target)], 
        eval_metric='auc', 
        verbose=0,
        early_stopping_rounds=EARLY_STOPPING_ROUNDS,
        #init_model=pretrain_lgbm,
        )
        
        
        temp_oof = model.predict_proba(val_df)[:, 1]
        temp_test = model.predict_proba(test[columns])[:, 1]

        train_oof[val_ind] = temp_oof
        test_preds += temp_test/num_folds
        
        aucs.append(roc_auc_score(val_target, temp_oof))
        
        print(f'Fold {f}: {roc_auc_score(val_target, temp_oof)}')
        
    print("Mean ROC AUC Score: ", np.mean(aucs))
    
    return train_oof, test_preds, np.mean(aucs)


In [ ]:
lgb_params={
    'learning_rate': 0.01,
    'metric': 'auc',
    'n_estimators': 10000,
    'num_leaves': 20067,
    'max_depth': 27,
    'reg_alpha': 9.630576598001266,
    'reg_lambda': 2.346945113164939,
    'colsample_bytree': 0.29858836720777177,
    'subsample': 0.6267448547447422,
    'min_child_samples': 61,
    'subsample_freq': 2,
    'subsample': 0.8329687190743886,
    'max_bin': 899,
    'min_data_per_group': 73,
    'random_state': SEED,
    'n_jobs': -1,
    'bagging_seed': SEED,
    'feature_fraction_seed': SEED
}


clf1 = LGBMClassifier(**lgb_params)

train_oof_1, test_preds_1, score_oof_1 = cross_valid(clf1, train, target, num_folds=NUM_FOLDS, random_state=SEED)

In [ ]:
print(f'Score oof: {score_oof_1:0.5f}') 

In [ ]:
clf2  = LGBMClassifier(
        n_estimators=12000,
        num_leaves=105,
        colsample_bytree=.8,
        subsample=.8,
        max_depth=7,
        reg_alpha=.1,
        reg_lambda=.1,
        min_split_gain=.01
    )

train_oof_2, test_preds_2, score_oof_2 = cross_valid(clf2, train, target, num_folds=NUM_FOLDS, random_state=SEED)

In [ ]:
print(f'Score oof: {score_oof_2:0.5f}') 

In [ ]:
clf3 = LGBMClassifier(
        n_estimators=12000,
        num_leaves=50,
        colsample_bytree=.8,
        subsample=.8,
        max_depth=8,
        reg_alpha=.1,
        reg_lambda=.1,
        min_split_gain=.01
    )

train_oof_3, test_preds_3, score_oof_3 = cross_valid(clf3, train, target, num_folds=NUM_FOLDS, random_state=SEED)

In [ ]:
print(f'Score oof: {score_oof_3:0.5f}') 

In [ ]:
tt1 = clf1.predict_proba(test)[:, 1]
tt2 = clf2.predict_proba(test)[:, 1]
tt3 = clf3.predict_proba(test)[:, 1]

In [ ]:
submission['target'] = (test_preds_1 + test_preds_2 + test_preds_3)/3 #(res['x'][0]*tt1+res['x'][1]*tt2+res['x'][2]*tt3)/3
submission.to_csv('submission.csv')
submission.head()


In [ ]:
submission['target'] =  (test_preds_1)#(res['x'][0]*tt1+res['x'][1]*tt2+res['x'][2]*tt3)/3
submission.to_csv('submission_model_1.csv')
submission.head()


In [ ]:
submission['target'] =  (test_preds_2) #(res['x'][0]*tt1+res['x'][1]*tt2+res['x'][2]*tt3)/3
submission.to_csv('submission_model_2.csv')
submission.head()

In [ ]:
submission['target'] =  (test_preds_3) #(res['x'][0]*tt1+res['x'][1]*tt2+res['x'][2]*tt3)/3
submission.to_csv('submission_model_3.csv')
submission.head()

# Some Visualizations

In [ ]:
features_num = ['cont0', 'cont1', 'cont2', 'cont3', 
                'cont4', 'cont5', 'cont6', 'cont7',
                'cont8', 'cont9', 'cont10']

# plot distribution of numerical features
for f in features_num:
    plt.figure(figsize=(8,4))
    train[f].plot(kind='hist', bins=100)
    plt.title(f)
    plt.grid()
    plt.show()


# Correlation

In [ ]:
import seaborn as sns
corr_pearson = train[features_num].corr(method='pearson')
corr_spearman = train[features_num].corr(method='spearman')

fig = plt.figure(figsize = (12,9))
sns.heatmap(corr_pearson, annot=True, cmap='RdYlGn', vmin=-1, vmax=+1)
plt.title('Pearson Correlation')
plt.show()

## Scatter plot

In [ ]:


# example of scatter plot - we pick pair having highest (Pearson) correlation
sns.jointplot(data=train, x='cont5', y='cont10',
              joint_kws = {'alpha': 0.1})
plt.show()



## Categorical Feature Plot

In [ ]:
features_cat = ['cat0', 'cat1', 'cat2', 'cat3',
                'cat4', 'cat5', 'cat6', 'cat7',
                'cat8', 'cat9']

# plot distribution of categorical features
for f in features_cat:
    plt.figure(figsize=(8,4))
    train[f].value_counts().plot(kind='bar')
    plt.title(f)
    plt.grid()
    plt.show()

Thank You